


### **COVID-19 Classification using pretrained VGG16 architecture**

**1. Mount google drive to download the images**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

**2. Importing required modules and libraries**

In [ ]:
from __future__ import print_function, division

import os
import zipfile
import random
import numpy as np
import time
import copy
import shutil

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable

import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt

**3. Using CUDA**

In [ ]:
use_gpu = torch.cuda.is_available()

**4. Download and Unzip the data file**

In [ ]:
drive_dir='/content/drive/MyDrive/DS-Data/'
zip_filename = 'Data.zip'

src_root_dir='Combined-Data'
trg_root_dir='Splitted-Data'
classes_dir = ['PNEUMONIA', 'NORMAL', 'COVID19']

DATA_PATH = trg_root_dir

In [ ]:
def data_download_unzip():
  local_zip = drive_dir + '/' + zip_filename
  zip_ref = zipfile.ZipFile(local_zip, 'r')
  zip_ref.extractall('/content')
  zip_ref.close()


In [ ]:
data_download_unzip()

**4.a Split data into train, test and validation**

In [ ]:
def train_val_test_split(val_ratio=0.15, test_ratio=0.15):
  
  !rm -rf Splitted-Data

  if not os.path.exists(trg_root_dir):
      os.makedirs(trg_root_dir)
 
  for cls in classes_dir:
    if not os.path.exists(trg_root_dir +'/train/' + cls):
      os.makedirs(trg_root_dir +'/train/' + cls)
    if not os.path.exists(trg_root_dir +'/val/' + cls):
      os.makedirs(trg_root_dir +'/val/' + cls)
    if not os.path.exists(trg_root_dir +'/test/' + cls):
      os.makedirs(trg_root_dir +'/test/' + cls)

    # Creating partitions of the data after shuffeling
    src = src_root_dir + '/' + cls # Folder to copy images from
    allFileNames = os.listdir(src)
  
    np.random.shuffle(allFileNames)
    train_FileNames, val_FileNames, test_FileNames = np.split(np.array(allFileNames),
                                                              [int(len(allFileNames)* (1 - val_ratio - test_ratio)), 
                                                               int(len(allFileNames)* (1 - test_ratio))])

    train_FileNames = [src+'/'+ name for name in train_FileNames.tolist()]
    val_FileNames = [src+'/' + name for name in val_FileNames.tolist()]
    test_FileNames = [src+'/' + name for name in test_FileNames.tolist()]
    print('Splitting data for the class : ', cls)
    print('Total images: ', len(allFileNames))
    print('Training: ', len(train_FileNames))
    print('Validation: ', len(val_FileNames))
    print('Testing: ', len(test_FileNames))

    # Copy-pasting images

    for name in train_FileNames:
        shutil.copy(name, trg_root_dir +'/train/' + cls)

    for name in val_FileNames:
        shutil.copy(name, trg_root_dir +'/val/' + cls)

    for name in test_FileNames:
        shutil.copy(name, trg_root_dir +'/test/' + cls)

In [ ]:
train_val_test_split(0.15, 0.15)

**5. Data augmentation and statistics**

In [ ]:
TRAIN = 'train'
VAL = 'val'
TEST = 'test'

data_transforms = {
  TRAIN: transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
  ]),

  VAL: transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
  ]),

  TEST: transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
  ])
}

image_datasets = {
  x: datasets.ImageFolder(
      os.path.join(DATA_PATH, x), 
      transform=data_transforms[x]
  )
  for x in [TRAIN, VAL, TEST]
}

dataloaders = {
  x: torch.utils.data.DataLoader(
      image_datasets[x], batch_size=8,
      shuffle=True, num_workers=4
  )
  for x in [TRAIN, VAL, TEST]
}
  
dataset_sizes = {x: len(image_datasets[x]) for x in [TRAIN, VAL, TEST]}

for x in [TRAIN, VAL, TEST]:
  print("Loaded {} images under {}".format(dataset_sizes[x], x))


class_names = image_datasets[TRAIN].classes
print("Classes: ", image_datasets[TRAIN].classes)

def get_count_metrics(folder, data_path=DATA_PATH):

    train_dir = os.path.join(data_path, folder)
    list_p = os.listdir(os.path.join(train_dir,'PNEUMONIA')) # dir is your directory path
    num_p = len(list_p)
    list_n = os.listdir(os.path.join(train_dir,'NORMAL')) # dir is your directory path
    num_n = len(list_n)
    list_c = os.listdir(os.path.join(train_dir,'COVID19')) # dir is your directory path
    num_c = len(list_c)
    count_tuple = (int(num_n), int(num_p), int(num_c))

    return count_tuple

print(get_count_metrics('train'))
print(get_count_metrics('test'))
print(get_count_metrics('val'))

**6. Visualize the dataset/images**

In [ ]:
def imshow(inp, title=None):
    inp = inp.numpy().transpose((1, 2, 0))
    plt.axis('off')
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)

def show_databatch(inputs, classes):
    out = torchvision.utils.make_grid(inputs)
    imshow(out, title=[class_names[x] for x in classes])

# Get a batch of training data
inputs, classes = next(iter(dataloaders[TRAIN]))
show_databatch(inputs, classes)


**7. Define the VGG16 model**
  

*   Setting the model pre-trained parameter to True
*   Customizing the last fully connected layer with 3 output classes
*   Setting requires_grad to false for all layers 





In [ ]:
vgg16 = models.vgg16_bn(pretrained=True)

# Freeze training for all layers
for param in vgg16.features.parameters():
    param.require_grad = False

num_features = vgg16.classifier[6].in_features
features = list(vgg16.classifier.children())[:-1] # Remove last layer
features.extend([nn.Linear(num_features, len(class_names))]) # Add our layer with 4 outputs
vgg16.classifier = nn.Sequential(*features) # Replace the model classifier

if use_gpu: 
  vgg16.cuda()

print(vgg16)

**7. Define loss function and optimizer**

In [ ]:
criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.SGD(vgg16.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

**8. Define funtion to evaluate test dataset and calculate accuracy**

In [ ]:
def eval_model(vgg, criterion):
    since = time.time()
    avg_loss = 0
    avg_acc = 0
    loss_test = 0
    acc_test = 0
    
    test_batches = len(dataloaders[TEST])
    print("Evaluating model. Total test batchs - ", test_batches)
    print('-' * 30)
    

    for i, data in enumerate(dataloaders[TEST]):
        if i % 100 == 0:
            print("\rTest batch {}/{} is in progress\n".format(i+1, test_batches), end='', flush=True)

        vgg.train(False)
        vgg.eval()

        inputs, labels = data
        if use_gpu:
            inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())
        else:
            inputs, labels = Variable(inputs), Variable(labels)

        outputs = vgg(inputs)
        _, preds = torch.max(outputs.data, 1)
        loss = criterion(outputs, labels)

        loss_test += loss.item()
        acc_test += torch.sum(preds == labels.data)
        del inputs, labels, outputs, preds
        torch.cuda.empty_cache()
        
    avg_loss = loss_test / dataset_sizes[TEST]
    avg_acc = acc_test / dataset_sizes[TEST]
    
    elapsed_time = time.time() - since
    print()
    print("Evaluation completed in {:.0f}m {:.0f}s".format(elapsed_time // 60, elapsed_time % 60))
    print("Avg loss (test): {:.4f}".format(avg_loss))
    print("Avg acc (test): {:.4f}".format(avg_acc))
    print('-' * 10)
    return avg_loss, avg_acc

**9. Test the model accuracy before training**

In [ ]:
#eval_model(vgg16, criterion)

**10. Define funtion to train the model with train data**

In [ ]:
def train_model(vgg, criterion, optimizer, scheduler, num_epochs=10):

    since = time.time()
    best_model_wts = copy.deepcopy(vgg.state_dict())
    
    best_acc = avg_loss = avg_acc = avg_loss_val = avg_acc_val = 0.0
   
    train_batches = len(dataloaders[TRAIN])
    val_batches = len(dataloaders[VAL])
    
    for epoch in range(num_epochs):
        print("Epoch {}/{} is in progress".format(epoch+1, num_epochs))
        print('-' * 30)
        
        loss_train = loss_val = acc_train = acc_val = 0.0

        vgg.train(True)

        for i, data in enumerate(dataloaders[TRAIN]):
            if i % 100 == 0:
                print("\rTraining batch {}/{} is in progress\n".format(i+1, train_batches), end='', flush=True)
                
            #Use half training dataset
            #if i >= train_batches / 2:
            #   break
                
            inputs, labels = data
            if use_gpu:
                inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())
            else:
                inputs, labels = Variable(inputs), Variable(labels)
            
            optimizer.zero_grad()
            outputs = vgg(inputs)
            
            _, preds = torch.max(outputs.data, 1)
            loss = criterion(outputs, labels)            
            loss.backward()
            optimizer.step()            
        
            loss_train += loss.item()
            acc_train += torch.sum(preds == labels.data)           
            del inputs, labels, outputs, preds
            torch.cuda.empty_cache()
        
        avg_loss = loss_train/dataset_sizes[TRAIN]
        avg_acc = acc_train/dataset_sizes[TRAIN]
              
        vgg.train(False)
        vgg.eval()
            
        for i, data in enumerate(dataloaders[VAL]):
            if i % 100 == 0:
                print("\rValidation batch {}/{} is in progress\n".format(i+1, val_batches), end='', flush=True)
                
            inputs, labels = data            
            if use_gpu:
                inputs, labels = Variable(inputs.cuda(), volatile=True), Variable(labels.cuda(), volatile=True)
            else:
                inputs, labels = Variable(inputs, volatile=True), Variable(labels, volatile=True)
            
            optimizer.zero_grad()            
            outputs = vgg(inputs)

            _, preds = torch.max(outputs.data, 1)
            loss = criterion(outputs, labels)

            loss_val += loss.item()
            acc_val += torch.sum(preds == labels.data)            
            del inputs, labels, outputs, preds
            torch.cuda.empty_cache()
        
        avg_loss_val = loss_val/dataset_sizes[VAL]
        avg_acc_val = acc_val/dataset_sizes[VAL]
        
        print()
        print("Epoch {} result: ".format(epoch))
        print("Avg loss (train): {:.4f}".format(avg_loss))
        print("Avg acc (train): {:.4f}".format(avg_acc))
        print("Avg loss (val): {:.4f}".format(avg_loss_val))
        print("Avg acc (val): {:.4f}".format(avg_acc_val))
        print('-' * 30)
        print()

        if avg_acc_val > best_acc:
            best_acc = avg_acc_val
            best_model_wts = copy.deepcopy(vgg.state_dict())
        
    elapsed_time = time.time() - since
    print()
    print("Training completed in {:.0f}m {:.0f}s".format(elapsed_time // 60, elapsed_time % 60))
    print("Best acc: {:.4f}".format(best_acc))
    
    vgg.load_state_dict(best_model_wts)
    return vgg, avg_loss_val, avg_acc_val



**11. Define funtion to predict classification for few images**

In [ ]:
def visualize_model(vgg, num_images=6):
    was_training = vgg.training
    
    # Set model for evaluation
    vgg.train(False)
    vgg.eval() 
    
    images_so_far = 0

    for i, data in enumerate(dataloaders[TEST]):
        inputs, labels = data
        size = inputs.size()[0]
        
        if use_gpu:
            inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())
        else:
            inputs, labels = Variable(inputs), Variable(labels)
        
        outputs = vgg(inputs)

        _, preds = torch.max(outputs.data, 1)
        predicted_labels = [preds[j] for j in range(inputs.size()[0])]
        
        print("Ground truth:")
        show_databatch(inputs.data.cpu(), labels.data.cpu())
        print("Prediction:")
        show_databatch(inputs.data.cpu(), predicted_labels)
        
        del inputs, labels, outputs, preds, predicted_labels
        torch.cuda.empty_cache()
        
        images_so_far += size
        if images_so_far >= num_images:
            break
        
    vgg.train(mode=was_training) # Revert model back to original training state

**12. Train the model on train dataset**

In [ ]:
vgg16, avg_loss_val, avg_acc_val = train_model(vgg16, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=20)

**13. Evaluae the model on test dataset**

In [ ]:
avg_loss_test, avg_acc_test = eval_model(vgg16, criterion)

**14. Plot accuracy and loss**

In [ ]:
#plt.figure(figsize=(10, 7))
#plt.plot(train_accuracy, color='green', label='train accuracy')
#plt.plot(val_accuracy, color='blue', label='validataion accuracy')
#plt.legend()
#plt.savefig('accuracy.png')
#plt.show()

**15. Predict classification for few images**

In [ ]:
visualize_model(vgg16, num_images=32)